In [5]:
#iort library
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import seaborn as sns

In [6]:
#import data 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
weather_train= pd.read_csv('weather_train.csv')
weather_test = pd.read_csv('weather_test.csv')
meta = pd.read_csv('building_metadata.csv')

In [7]:
meta

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN
...,...,...,...,...,...,...
1444,15,1444,Entertainment/public assembly,19619,1914.0,NaN
1445,15,1445,Education,4298,NaN,NaN
1446,15,1446,Entertainment/public assembly,11265,1997.0,NaN
1447,15,1447,Lodging/residential,29775,2001.0,NaN


In [8]:
print('train shape is {}'.format(train.shape))
print('test shape is {}'.format(test.shape))
print('weather_train shape is {}'.format(weather_train.shape))
print('weather_test shape is {}'.format(weather_test.shape))
print('metadata shape is {}'.format(meta.shape))

train shape is (20216100, 4)
test shape is (41697600, 4)
weather_train shape is (139773, 9)
weather_test shape is (277243, 9)
metadata shape is (1449, 6)


Reduce memory 

In [9]:
#Function to reduce memory size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [10]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
weather_train = reduce_mem_usage(weather_train)
weather_test = reduce_mem_usage(weather_test)
meta= reduce_mem_usage(meta)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [ ]:
def description(df):
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary

In [ ]:
description(train)

In [ ]:
description(weather_train)

In [ ]:
description(meta)

merge data

In [11]:
#merge data 
train = train.merge(meta, on='building_id', how='left')
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')
test =test.merge(meta, on='building_id', how='left')
test =test.merge(weather_test, on=['site_id', 'timestamp'], how='left')

In [ ]:
plt.figure(figsize = (18,12))
plt.suptitle('Meter Reading Values Distribution', fontsize = 22)
plt.subplot(221)
sns.distplot(train['meter_reading'], hist = False,kde_kws={"shade": True})
plt.title('Meter Reading Distribution', fontsize = 18)
plt.xlabel("")
plt.ylabel('Probability', fontsize = 15)

plt.subplot(222)
sns.distplot(np.log1p(train['meter_reading']), hist = False,kde_kws={"shade": True})
plt.title('Meter Reading (Log1p) Distribution', fontsize = 18)
plt.xlabel("")
plt.ylabel("Probability",fontsize = 15)

plt.subplot(223)
g = sns.countplot(train['meter'])
plt.xticks(np.arange(4),labels =['Electricity','Chilledwater','Steam','Hotwater'])
g.set_title('Meter Type Countplot', fontsize = 18)
g.set_xlabel('Type', fontsize = 15)
g.set_ylabel('Count', fontsize = 15)

tmp0 = train.loc[train['meter'] == 0]['meter_reading'].max()
tmp1 = train.loc[train['meter'] == 1]['meter_reading'].max()
tmp2 = train.loc[train['meter'] == 2]['meter_reading'].max()
tmp3 = train.loc[train['meter'] == 3]['meter_reading'].max()

tmp_meter = ['Electricity','Chilledwater','Steam','Hotwater']
tmp_meter_reading = [tmp0, tmp1, tmp2, tmp3]

tmp = pd.DataFrame(tmp_meter,tmp_meter_reading)
tmp = tmp.reset_index()
tmp.columns = ['meter','max_meter_reading']

gt = g.twinx()
gt = sns.pointplot(x = 'max_meter_reading', y = 'meter', data = tmp, color = 'black',
                  order = ['Electricity','Chilledwater','Steam','Hotwater'], legend = False)
gt.set_ylabel('Max Meter Reading by Meter Type',fontsize = 15)

plt.subplot(224)
sns.boxplot(x= 'meter', y = 'meter_reading', data = train)
plt.xticks(np.arange(4),labels =['Electricity','Chilledwater','Steam','Hotwater'])
plt.title('Meter Reading Distribution by Meter Type', fontsize = 18)
plt.xlabel('Meter Type', fontsize = 15)
plt.ylabel('Meter Reading Values', fontsize = 15)

plt.subplots_adjust(hspace = 0.85, top = .9)
plt.show()

In [13]:
description(train)

,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,building_id,int16,0,1449,0,1,2
1,meter,int8,0,4,0,0,0
2,timestamp,object,0,8784,2016-01-01 00:00:00,2016-01-01 00:00:00,2016-01-01 00:00:00
3,meter_reading,float32,0,1688175,0.0,0.0,0.0
4,site_id,int8,0,16,0,0,0
5,primary_use,object,0,16,Education,Education,Education
6,square_feet,int32,0,1397,7432,2720,5376
7,year_built,float16,12127645,116,2008.0,2004.0,1991.0
8,floor_count,float16,16709167,18,NaN,NaN,NaN
9,air_temperature,float16,96658,619,25.0,25.0,25.0


In [14]:
description(test)

,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,row_id,int32,0,41697600,0,1,2
1,building_id,int16,0,1449,0,1,2
2,meter,int8,0,4,0,0,0
3,timestamp,object,0,17520,2017-01-01 00:00:00,2017-01-01 00:00:00,2017-01-01 00:00:00
4,site_id,int8,0,16,0,0,0
5,primary_use,object,0,16,Education,Education,Education
6,square_feet,int32,0,1397,7432,2720,5376
7,year_built,float16,24598080,116,2008.0,2004.0,1991.0
8,floor_count,float16,34444320,18,NaN,NaN,NaN
9,air_temperature,float16,221901,639,17.796875,17.796875,17.796875


In [15]:
train.head(22)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
5,5,0,2016-01-01 00:00:00,0.0,0,Education,8000,2000.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
6,6,0,2016-01-01 00:00:00,0.0,0,Lodging/residential,27926,1981.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
7,7,0,2016-01-01 00:00:00,0.0,0,Education,121074,1989.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
8,8,0,2016-01-01 00:00:00,0.0,0,Education,60809,2003.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
9,9,0,2016-01-01 00:00:00,0.0,0,Office,27000,2010.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [ ]:
 Feature Engineering

In [16]:
train["timestamp"] = pd.to_datetime(train["timestamp"])
train["hour"] = np.uint8(train["timestamp"].dt.hour)
train["day"] = np.uint8(train["timestamp"].dt.day)
train["weekday"] = np.uint8(train["timestamp"].dt.weekday)
train["month"] = np.uint8(train["timestamp"].dt.month)

test["timestamp"] = pd.to_datetime(test["timestamp"])
test["hour"] = np.uint8(test["timestamp"].dt.hour)
test["day"] = np.uint8(test["timestamp"].dt.day)
test["weekday"] = np.uint8(test["timestamp"].dt.weekday)
test["month"] = np.uint8(test["timestamp"].dt.month)

In [17]:
train['meter'].replace({0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"},inplace=True)
test['meter'].replace({0:"Electricity",1:"ChilledWater",2:"Steam",3:"HotWater"},inplace=True)

In [18]:
train['year_built'] = np.uint8(train['year_built']-1900)
test['year_built'] = np.uint8(test['year_built']-1900)

In [19]:
train['year_built']

0           108
1           104
2            91
3           102
4            75
           ... 
20216095     14
20216096      0
20216097     97
20216098    101
20216099    101
Name: year_built, Length: 20216100, dtype: uint8

Imuput missing values

In [20]:
Feature_with_NA = train.loc[:, ['floor_count','air_temperature', 'cloud_coverage', 'dew_temperature',
                      'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction','wind_speed']]
for i in Feature_with_NA.columns:
    train[i].fillna(train[i].median(), inplace = True)
for i in Feature_with_NA.columns:
    test[i].fillna(test[i].median(), inplace = True)

In [ ]:
#for i in Feature_with_NA.columns:
   # train[i].fillna(train[i].means(), inplace = True)
#for i in Feature_with_NA.columns:
#    test[i].fillna(test[i].mean(), inplace = True)

In [21]:
from sklearn.preprocessing import LabelEncoder

train['primary_use'] = LabelEncoder().fit_transform(train['primary_use'])
test['primary_use'] = LabelEncoder().fit_transform(test['primary_use'])

In [22]:
test['primary_use']

0           0
1           0
2           0
3           0
4           0
           ..
41697595    1
41697596    0
41697597    1
41697598    4
41697599    6
Name: primary_use, Length: 41697600, dtype: int64

In [23]:
train_meter_encodage  = pd.get_dummies(train['meter'])
test_meter_encodage = pd.get_dummies(test['meter'])

In [24]:
train.drop(columns=['meter', 'timestamp'], axis = 1, inplace = True)
test.drop(columns=['meter', 'timestamp'], axis = 1, inplace = True)

In [25]:
train_x = pd.concat([train_meter_encodage , train], axis = 1)
test_x = pd.concat([test_meter_encodage , test], axis = 1)

In [27]:
display(test_x.head(3))

,ChilledWater,Electricity,HotWater,Steam,row_id,building_id,site_id,primary_use,square_feet,year_built,...,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month
0,0,1,0,0,0,0,0,0,7432,108,...,4.0,11.703125,0.0,1021.5,100.0,3.599609,0,1,6,1
1,0,1,0,0,1,1,0,0,2720,104,...,4.0,11.703125,0.0,1021.5,100.0,3.599609,0,1,6,1
2,0,1,0,0,2,2,0,0,5376,91,...,4.0,11.703125,0.0,1021.5,100.0,3.599609,0,1,6,1


In [28]:
target = train_x['meter_reading']
train = train_x.drop(['meter_reading'], axis = 1)
test = test_x.drop(['row_id'], axis = 1)

In [29]:
print('Input matrix dimension:', train.shape)
print('Output vector dimension:',target.shape)
print('Test data dimension:', test.shape)

Input matrix dimension: (20216100, 21)
Output vector dimension: (20216100,)
Test data dimension: (41697600, 21)


In [30]:
from sklearn.tree import DecisionTreeRegressor
tree_model = DecisionTreeRegressor(min_samples_split = 20, min_samples_leaf = 10)
tree_model = tree_model.fit(train, target)

In [31]:
y_pred = tree_model.predict(test)

In [28]:
#y1=pd.read_csv("submission.csv")

In [32]:
my_submission = pd.DataFrame({'row_id': test_x.row_id, 'meter_reading': y_pred})
my_submission.to_csv('submission3.csv', index=False)

In [33]:
!tar cvfz tt.tar.gz submission3.csv

submission3.csv
